> # **Introduction**
> 
> So in this notebook, I shall be explaining and implementing classical stringmatching algorithms to measure the similairty score between 2 words.  
>   
> We shall first be using NLP techniques to preprocess and prepare the data, afterwhich we shall be implementing the following algorithms:  
>   
> 1) cosine similarity 
> 
> 2) jaccard index
>
> 3) levenshtein distance 
>
> 4) euclidean distance


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# - Reading the Data and Data Preprocessing

> **In This segment I shall be importing all necessary libraries and preprocessing the data before we compute our similarty measures**.
>
> **I have used a number of libraries for preprocessing including NLTK, Textacy and gensim among others. You shall have to install Textacy as it is not pre installed on Kaggle.**

In [2]:
# DOWNLOADING TEXTACY

!pip install textacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.8 MB/s eta 0:00:00


In [3]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


* > **Let us import the rest of the libraries.**

In [4]:
!pip install --upgrade textacy spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# IMPORTING NECESSARY LIBRARIES

from textacy import preprocessing
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from gensim.models import Word2Vec
import gensim

* > **Lets start, by first reading and exploring the data!**

In [6]:
# READING THE DATA

data = pd.read_csv("train.csv")
data.head(5)

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


* > **Since in this notebook we are only interested in leaning about classical similarity measures, there is no point keeping some columns, better remove them!**

In [7]:
# DROPPING SOME COLUMNS

data.drop(columns = ['id', 'context'], inplace = True)

In [8]:
# REVIEW DATA 

data.head(5)

,anchor,target,score
0,abatement,abatement of pollution,0.50
1,abatement,act of abating,0.75
2,abatement,active catalyst,0.25
3,abatement,eliminating process,0.50
4,abatement,forest region,0.00


*  > **In this segment we have used NLP techniques to clean up the data prior to implementing the similarity measures.**
>  
*  > **First it is important to ensure that all words have the same case so that vector representations can be same, "A" and "a" might be the same letter but not the same vector, thus we use lowercase() function below to change the words to a coomon case.**

In [9]:
# CONVERTING TEXT TO LOWER CASE

data["anchor"] = data["anchor"].str.lower()
data["target"] = data["target"].str.lower()

* > **We shall also remove the stopwords from the columns which refers to removing common words like 'the', 'a', which do not add much semantic meaning to the text.**

In [10]:
# REMOVING THE STOPWORDS
nltk.download('stopwords')
stop = stopwords.words('english')

data["anchor"] = data['anchor'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data["target"] = data['target'].apply(lambda y: ' '.join([word for word in y.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


* > **Whitespaces, Hashtags and punctuations do not add any semantic meaning to the text but occupy vector representations, thus it is necessary to remove them.** 

In [11]:
# REMOVING WHITESPACE, HASHTAGS. HTML TAGS AND PUNCTUATION

preproc = preprocessing.make_pipeline(
    preprocessing.remove.punctuation,
    preprocessing.normalize.whitespace,
    preprocessing.replace.hashtags,
    preprocessing.remove.html_tags
 )

data["anchor"] = data["anchor"].apply(preproc)
data["target"] = data["target"].apply(preproc)

* > **Let's look at our data after the initial preprocessing.**

In [12]:
# REVIEW THE DATA

data.head()

,anchor,target,score
0,abatement,abatement pollution,0.50
1,abatement,act abating,0.75
2,abatement,active catalyst,0.25
3,abatement,eliminating process,0.50
4,abatement,forest region,0.00


* > **In the below cell we shall implement stemming which is a method of reducing the words to their root stem, thus making it easier for later comparisons.**

In [13]:
# STEMMING THE WORDS USING SNOWBALL STEMMER

stemmer = SnowballStemmer("english")
data['anchor'] = data['anchor'].apply(lambda x: stemmer.stem(x))
data['target'] = data['target'].apply(lambda x: stemmer.stem(x))

* > **Let's see how our data looks after Stemming.**

In [14]:
# REVIEW THE DATA

data.head(5)

,anchor,target,score
0,abat,abatement pollut,0.50
1,abat,act ab,0.75
2,abat,active catalyst,0.25
3,abat,eliminating process,0.50
4,abat,forest region,0.00


* > **We shall also be converting our Text into Vectors using TF-IDF algorithm, this is necessary as some similarity measures compare the distance between vectors.**

In [15]:
# CONVERTING TEXT TO VECTORS USING TF-IDF ALGORITHM

tf = TfidfVectorizer(analyzer = 'char')

anchor_tf = tf.fit_transform(data['anchor']).toarray()
target_tf = tf.transform(data['target']).toarray()

* > **Let's check the shape of the resulting vectors.**

In [16]:
# CHECKING SHAPE OF THE VECTORS FORMED USING TF-IDF

print(anchor_tf.shape)
print(target_tf.shape)

(36473, 30)
(36473, 30)


* > **Now that we have completed our preprocessing, let's move on to computing the similarity measures.**

># **1) COSINE SIMILARITY**
>   
> * Cosine similarity is used to compute the similarity between 2 vectors, wherein the resultant score is the cosine of the angle between the 2 vectors in multi dimensional space.
>   
> * So the similarity measure is computed by taking a dot product of the 2 vectors and then dividing the dot product by the product of the magnitudes of the vectors.  
> 
> * The advantage it offers above other measures like Euclidean distance is that it is calculates the orientation of vectors with respect to each other and not the magnitude so incase vectors have varied sizes, Euclidean distance shall show a high seperation between the vectors but Cosine similarity can still be less.


In [17]:
from scipy import spatial

cos_sim_vals  = []

for a, t in zip(anchor_tf, target_tf):
    similarity = 1 - spatial.distance.cosine(a, t)
    cos_sim_vals.append(similarity)

/usr/local/lib/python3.9/dist-packages/scipy/spatial/distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [18]:
data["Cosine Similarity"] = cos_sim_vals

> * **Let's review the measure by printing the data.*

In [19]:
data.head()

,anchor,target,score,Cosine Similarity
0,abat,abatement pollut,0.50,0.616771
1,abat,act ab,0.75,0.894857
2,abat,active catalyst,0.25,0.494009
3,abat,eliminating process,0.50,0.161542
4,abat,forest region,0.00,0.071977


># **2) LEVENSHTEIN DISTANCE**
>   
> 
> * Levenshtein distance is a popular stringmatching algorithm that tells us how much change shall be required to made to a text in order to transform it to the other.
> 
> * More simply it measures how different 2 words are from each other.

In [20]:
!pip install python-Levenshtein
from Levenshtein import distance as lev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.3 MB/s eta 0:00:00


In [21]:
from Levenshtein import ratio

> * **Let;s compute the Levenshtein Distance and store it as a list.**

In [22]:
lev_sim_vals  = []

for a, t in zip(data["anchor"], data["target"]):
    lev_similarity = ratio(a, t)
    lev_sim_vals.append(lev_similarity)

> * **We can now add the list to the data**

In [23]:
data["Levenshtein Ratio"] = lev_sim_vals

In [24]:
data.head(5)

,anchor,target,score,Cosine Similarity,Levenshtein Ratio
0,abat,abatement pollut,0.50,0.616771,0.400000
1,abat,act ab,0.75,0.894857,0.400000
2,abat,active catalyst,0.25,0.494009,0.315789
3,abat,eliminating process,0.50,0.161542,0.173913
4,abat,forest region,0.00,0.071977,0.117647


># **3) JACCARD SCORE**
>   
> * The Jaccard score is a popular score used to measure the similarity between 2 vectors/sequences.
> 
> * It has a score ranging from 0 to 1, higher the score more the similarity
> 
> * It is mathematically computed as the intersection of items present in both sequences divided by the union of items present in both. It's applications are vast, and I have even used in Link Prediction tasks in Network Science ;)

In [25]:
!pip install Distance

import distance

distance.jaccard("decide", "resize")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Distance: filename=Distance-0.1.3-py3-none-any.whl size=16275 sha256=7a72bce82119928a3cef99c29ebca77420f89bf764ee1fdd243b5010cfcb7456
  Stored in directory: /root/.cache/pip/wheels/fb/b3/aa/04241cced6d1722b132273b1d6aafba317887ec004f48b853a
Successfully built Distance


0.7142857142857143

> * **We shall compute the Jaccard Index and store it in a list.**

In [26]:
jac_sim_vals  = []

for a, t in zip(data["anchor"], data["target"]):
    jac_sim = distance.jaccard(a, t)
    jac_sim_vals.append(jac_sim)

> * **Now let's add the list as a new column to the data.**

In [27]:
data["Jaccard Similarity"] = jac_sim_vals

> * **Let's reveiew our resulting data**

In [28]:
data.head(5)

,anchor,target,score,Cosine Similarity,Levenshtein Ratio,Jaccard Similarity
0,abat,abatement pollut,0.50,0.616771,0.400000,0.727273
1,abat,act ab,0.75,0.894857,0.400000,0.400000
2,abat,active catalyst,0.25,0.494009,0.315789,0.818182
3,abat,eliminating process,0.50,0.161542,0.173913,0.866667
4,abat,forest region,0.00,0.071977,0.117647,0.916667


>#  **4) EUCLIDEAN DISTANCE**
> 
> * Euclidean distance is used to measure the shortest distance between 2 points in an n dimensional space.
>  
> * It is more simply the L2 norm between the vectors in the n dimensional space. 
>
> * This is the most classical method used for similarity measurement, but carries alot of limitations.

In [29]:
from sklearn.preprocessing import MinMaxScaler

> * **Lets compute the euclidean distance first and store it as a list.**

In [30]:
euc_sim_vals = []
for a, t in zip(anchor_tf, target_tf):
    dist = np.linalg.norm(a-t)
    euc_sim_vals.append(dist)

In [31]:
len(euc_sim_vals)

36473

> * **We shall need to convert the list to an array for furhte steps.**

In [32]:
euc_sim_vals = np.array(euc_sim_vals)

In [33]:
euc_sim_vals

array([0.87547602, 0.45856902, 1.00597316, ..., 0.77594036, 0.64808072,
       0.86214448])

> * **Let's normalize the distance to bring it between 0 and 1.**

In [34]:
ss = MinMaxScaler()
normalized_euc= ss.fit_transform(euc_sim_vals.reshape(-1,1))

In [35]:
len(normalized_euc)

36473

In [36]:
data["Normalized Eucliden"] = normalized_euc

> * **Let's finally review our measures.** 

In [37]:
data.head(5)

,anchor,target,score,Cosine Similarity,Levenshtein Ratio,Jaccard Similarity,Normalized Eucliden
0,abat,abatement pollut,0.50,0.616771,0.400000,0.727273,0.619055
1,abat,act ab,0.75,0.894857,0.400000,0.400000,0.324257
2,abat,active catalyst,0.25,0.494009,0.315789,0.818182,0.711330
3,abat,eliminating process,0.50,0.161542,0.173913,0.866667,0.915674
4,abat,forest region,0.00,0.071977,0.117647,0.916667,0.963339
